In [13]:
import glob
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import os
import copy
from PIL import Image
import datetime
from tqdm import tqdm_notebook
import torchvision.transforms as transforms



In [28]:
#reading filename lists
num_patients = 125
window_size = [256,256,256]
target_resolution = [1,1,1] # w x h x d (looking down the sagittal-caronal axis)

CT_file_list = sorted(glob.glob("../scratch_alan/prostate_jc/PROSTATE*/CT.bin"))[:num_patients]
prostate_file_list = sorted(glob.glob("../scratch_alan/prostate_jc/PROSTATE*/Prostate*.bin"))[:num_patients]
info_file_list = sorted(glob.glob("../scratch_alan/prostate_jc/PROSTATE*/info.txt"))[:num_patients]
info2_file_list = sorted(glob.glob("../scratch_alan/prostate_jc/PROSTATE*/info_2.txt"))[:num_patients]
#saving options
target_save_path = "./datasets/prostate_segmentation/"
slices_per_patient = 30

In [29]:
len(prostate_file_list)

111

In [30]:
ct = []
for x in CT_file_list:
    ct.append(int(x.split('../scratch_alan/prostate_jc/PROSTATE')[1].split('/')[0]))
ct = set(ct)
info2 = []
for x in CT_file_list:
    info2.append(int(x.split('../scratch_alan/prostate_jc/PROSTATE')[1].split('/')[0]))
info2 = set(ct)
info = []
for x in CT_file_list:
    info.append(int(x.split('../scratch_alan/prostate_jc/PROSTATE')[1].split('/')[0]))
info = set(ct)
prostate = []
for x in prostate_file_list:
    prostate.append(int(x.split('../scratch_alan/prostate_jc/PROSTATE')[1].split('/')[0]))
prostate = set(prostate)

t = ct.intersection(prostate).intersection(info).intersection(info2)

In [31]:
ct_file = []
for x in CT_file_list:
    if int(x.split('../scratch_alan/prostate_jc/PROSTATE')[1].split('/')[0]) in t:
        if len(x.split('../scratch_alan/prostate_jc/PROSTATE')[1].split('/')[0]) > 2:
            ct_file.append('../scratch_alan/prostate_jc/PROSTATE'+x.split('../scratch_alan/prostate_jc/PROSTATE')[1].split('/')[0]+'/CT.bin')
        elif len(x.split('../scratch_alan/prostate_jc/PROSTATE')[1].split('/')[0]) == 2:
            ct_file.append('../scratch_alan/prostate_jc/PROSTATE0'+x.split('../scratch_alan/prostate_jc/PROSTATE')[1].split('/')[0]+'/CT.bin')
        elif len(x.split('../scratch_alan/prostate_jc/PROSTATE')[1].split('/')[0]) == 1:
            ct_file.append('../scratch_alan/prostate_jc/PROSTATE0'+x.split('../scratch_alan/prostate_jc/PROSTATE')[1].split('/')[0]+'/CT.bin')

prostate_file = []
for x in prostate_file_list:
    if int(x.split('../scratch_alan/prostate_jc/PROSTATE')[1].split('/')[0]) in t:
        if len(x.split('../scratch_alan/prostate_jc/PROSTATE')[1].split('/')[0]) > 2:
            prostate_file.append('../scratch_alan/prostate_jc/PROSTATE'+x.split('../scratch_alan/prostate_jc/PROSTATE')[1].split('/')[0]+'/Prostate.bin')
        elif len(x.split('../scratch_alan/prostate_jc/PROSTATE')[1].split('/')[0]) == 2:
            prostate_file.append('../scratch_alan/prostate_jc/PROSTATE0'+x.split('../scratch_alan/prostate_jc/PROSTATE')[1].split('/')[0]+'/Prostate.bin')
        elif len(x.split('../scratch_alan/prostate_jc/PROSTATE')[1].split('/')[0]) == 1:
            prostate_file.append('../scratch_alan/prostate_jc/PROSTATE0'+x.split('../scratch_alan/prostate_jc/PROSTATE')[1].split('/')[0]+'/Prostate.bin')

info_file = []
for x in info_file_list:
    if int(x.split('../scratch_alan/prostate_jc/PROSTATE')[1].split('/')[0]) in t:
        if len(x.split('../scratch_alan/prostate_jc/PROSTATE')[1].split('/')[0]) > 2:
            info_file.append('../scratch_alan/prostate_jc/PROSTATE'+x.split('../scratch_alan/prostate_jc/PROSTATE')[1].split('/')[0]+'/info.txt')
        elif len(x.split('../scratch_alan/prostate_jc/PROSTATE')[1].split('/')[0]) == 2:
            info_file.append('../scratch_alan/prostate_jc/PROSTATE0'+x.split('../scratch_alan/prostate_jc/PROSTATE')[1].split('/')[0]+'/info.txt')
        elif len(x.split('../scratch_alan/prostate_jc/PROSTATE')[1].split('/')[0]) == 1:
            info_file.append('../scratch_alan/prostate_jc/PROSTATE0'+x.split('../scratch_alan/prostate_jc/PROSTATE')[1].split('/')[0]+'/info.txt')

info_file2 = []
for x in info_file_list:
    if int(x.split('../scratch_alan/prostate_jc/PROSTATE')[1].split('/')[0]) in t:
        if len(x.split('../scratch_alan/prostate_jc/PROSTATE')[1].split('/')[0]) > 2:
            info_file2.append('../scratch_alan/prostate_jc/PROSTATE'+x.split('../scratch_alan/prostate_jc/PROSTATE')[1].split('/')[0]+'/info_2.txt')
        elif len(x.split('../scratch_alan/prostate_jc/PROSTATE')[1].split('/')[0]) == 2:
            info_file2.append('../scratch_alan/prostate_jc/PROSTATE0'+x.split('../scratch_alan/prostate_jc/PROSTATE')[1].split('/')[0]+'/info_2.txt')
        elif len(x.split('../scratch_alan/prostate_jc/PROSTATE')[1].split('/')[0]) == 1:
            info_file2.append('../scratch_alan/prostate_jc/PROSTATE0'+x.split('../scratch_alan/prostate_jc/PROSTATE')[1].split('/')[0]+'/info_2.txt')

In [32]:
from scipy.ndimage import zoom
def match_images_res_single(vol_1,res_1,target):
    rescale_1 = res_1/target
    if rescale_1[0]!=1 or rescale_1[1]!=1 or rescale_1[2]!=1:
        vol_1 = zoom(vol_1, (rescale_1))
        
    return(vol_1)

In [33]:

CT_file_list = ct_file[:num_patients]
prostate_file_list = prostate_file[:num_patients]

info_file_list = info_file[:num_patients]
info2_file_list = info_file2[:num_patients]

array_list = []
p_list = []
size_list = []   

In [34]:
# Define functions to load data with different format
def CT_process(file_list,index):

    array_list = []
    for i in tqdm_notebook(range(len(file_list))):
        with open(file_list[i],'rb') as file:
            with open(info_file_list[i + 5*index],'r') as info_file:
                with open(info2_file_list[i + 5*index],'r') as info2_file:
                    binary = np.frombuffer(file.read(),dtype=np.int16)
                    array = np.array(binary, dtype=np.int16)
                    data_shape = [int(x.strip()) for x in info_file.readlines()[0].split()]
    #                 print(array.shape, np.prod(np.array(data_shape)))                
                    array = np.reshape((array),data_shape[::-1]*np.array(target_resolution)[::-1]) # Normalize the CT images to 0-1
                    #resolution = [float(x.strip()) for x in info2_file.readlines()[0].split()]
                    #array = match_images_res_single(array,np.array(resolution),np.array(target_resolution)[::-1])
    #                 print(array.shape)
    #                 print(array[:,250,:])
    

                    
                    new_array = np.transpose(array, (1, 2, 0)) # The input is h*d*w, transpose it to d*h*w
            
                    
                    
                    #print(resolution)
    #                 print(new_array[250,:,:])
                    array_list.append(new_array)
    return array_list


def input_process(file_list,index):
    array_list = []

    for i in tqdm_notebook(range(len(file_list))):
        with open(file_list[i],'rb') as file:
            with open(info_file_list[i + 5*index],'r') as info_file:
                with open(info2_file_list[i + 5*index],'r') as info2_file:
                    binary = np.frombuffer(file.read(),dtype=np.int8)
                    array = np.array(binary, dtype=np.int8)
                    data_shape = [int(x.strip()) for x in info_file.readlines()[0].split()]
                    array = np.reshape((array),data_shape[::-1]*np.array(target_resolution)[::-1]) # Normalize the CT images to 0-1
                    #normalize
                    #normalize      

                    new_array = np.transpose(array, (1,2,0)) # The input is d , transpose it to h w d
                
                      
                    array_list.append(new_array)

    return array_list


In [35]:
# Show images for each patient
def show_image(patient_id, dimention = 85):
    fig = plt.figure(figsize=[25,10])
    fig.suptitle(f"Patient: {patient_id}", fontsize=25)
    ax = plt.subplot("221")
    ax.set_title("CT",fontsize=16)
    ax.imshow(CT[patient_id][:,:,dimention])

    ax = plt.subplot("222")
    ax.set_title("Prostate",fontsize=16)
    ax.imshow(labels[patient_id][:,:,dimention])
    
    plt.show()
    # Show images for each patient
def show_label(patient_id, dimention = 85):
    fig = plt.figure(figsize=[25,10])
    fig.suptitle(f"Patient: {patient_id}", fontsize=25)
    ax = plt.subplot("221")
    ax.set_title("CT",fontsize=16)
    ax.imshow(labels[patient_id][:,:,dimention])

    ax = plt.subplot("222")
    ax.set_title("Prostate",fontsize=16)
    ax.imshow(labels[patient_id][:,:,dimention])
    
    plt.show()
def input_get(i, channel1, img_for_center):
    box1 = box_around(channel1[i], center_of_mass(img_for_center[i]), window_size)[0,0,:,:,:]
    return box1

In [36]:
def center_of_mass(target, shift=None):
    target = np.squeeze(target)
    assert len(target.shape) == 3
    shift = [shift]*3 if isinstance(shift, (int,float)) else shift if isinstance(shift, (tuple,list)) else (0,0,0)
    # shift = np.concatenate(([0]*(len(target.shape[-3:])-3), np.random.randn(3) * shift))
    shift = np.random.randn(3) * shift
    return np.array(shift + [np.mean(d) for d in np.where(target>0.5)], dtype=int)


def box_around(image, center, box_size, return_corner=False, pad=True):
    slices = [slice(None)] * 5
    corner = [None] * 5
    padding = [(0,0)] * 5
    for i, (sha, siz, cen) in enumerate(zip(reversed(np.shape(image)), reversed(box_size), reversed(center))):
        if siz is not None:
            ovlp = max(0, int(cen + siz / 2. - sha))
            udlp = max(0, -int(cen - siz / 2))
            left = udlp + int(cen - siz / 2.) - ovlp
            padding[-i - 1] = max(0, int(np.floor(-left / 2))), max(0, int(np.ceil(-left / 2)))
            corner[-i - 1] = left  # max(left, padding[-i-1][0]) #udlp + int(cen-siz/2.) - ovlp
            slices[-i - 1] = slice(max(0, left), left + siz)

    ret = copy.deepcopy(reshape_image(image)[slices])
    if any(pad != (0,0) for pad in padding): ret = np.pad(ret, padding, 'constant', constant_values=np.min(ret))
    if return_corner: return ret, corner[-3:]
    return ret
def reshape_image(image):
    shape = [1]*(5-len(image.shape)) + list(image.shape)
    return image.reshape(shape)

In [37]:
len(prostate_file)

111

In [40]:
import random
import shutil
import numpy as np

trainfolder_a = os.path.join('datasets/prostate_segmentation/train_A')
trainfolder_b = os.path.join('datasets/prostate_segmentation/train_B')
testfolder_a = os.path.join('datasets/prostate_segmentation/test_A')
testfolder_b = os.path.join('datasets/prostate_segmentation/test_B')
for i in range(20): #split up data

    labels = input_process(prostate_file[i*5:(i+1)*5],i)
    CT = CT_process(ct_file[i*5:(i+1)*5],i)
    for patient_index in range(5):
        #get pictures
        temp_inx = 5*i + patient_index
        if temp_inx !=4 and temp_inx!=7:

            current_CT = box_around(CT[patient_index] , center_of_mass(labels[patient_index]), window_size)[0,0,:,:,:]
            current_prostate = box_around(labels[patient_index] , center_of_mass(labels[patient_index]), window_size)[0,0,:,:,:]

            z_dim = current_CT.shape[2]
            has_mass = []
            has_no_mass = []
            for z_pixel in range(z_dim):
                #find which depths has prostate mass
                if current_prostate[:,:,z_pixel].sum() > 0:

                    has_mass.append(z_pixel)
                else:
                    has_no_mass.append(z_pixel)
                    
            #get pdf for sampling with mass
            p_has_mass = []
            tote_vol = 0
            for d in has_mass:
                curr_slice = current_prostate[:,:,d]
                curr_p =  curr_slice.shape[0] * curr_slice.shape[1] - curr_slice.sum()
                tote_vol += curr_p
                p_has_mass.append(curr_p)
            p_has_mass = np.array([p/tote_vol for p in p_has_mass])    
            has_mass = np.array(has_mass)
            
            
            selected_mass = np.random.choice(has_mass,size = int(slices_per_patient/2),p=p_has_mass )
            selected_no_mass = random.sample(has_no_mass,int(slices_per_patient/2))
            current_slice = 0
            for current_mass_slice in range(len(selected_mass)):   
                
                slice_a = current_CT[:,:,selected_mass[current_mass_slice]]
                
                slice_b = current_prostate[:,:,selected_mass[current_mass_slice]]


                #Normalizing
                #slice_a = (slice_a-slice_a.mean()+0.5)/(  slice_a.std())
                #slice_b = (slice_b-slice_b.mean()+0.5)/(  slice_b.std())

                slice_a = slice_a/slice_a.max()*255
                slice_b = slice_b*255
                

                img_a = Image.fromarray(slice_a) # slice images
                img_b = Image.fromarray(slice_b)    
                
                assert(img_a.size == img_b.size)
                aligned_image = Image.new("RGB", (img_a.size[0] * 2, img_a.size[1]))
                aligned_image.paste(img_a, (0, 0))
                aligned_image.paste(img_b, (img_a.size[0], 0))

                if (temp_inx)%10 in [0,1,2,3,4,5,6,7]:
                    aligned_image.save(os.path.join(target_save_path,'train_new', 'patient_{:04d}'.format(temp_inx)+'_slice_{:04d}'.format(current_slice)+'.jpg'))
                elif (temp_inx)%10 == 8:
                    aligned_image.save(os.path.join(target_save_path,'val_new', 'patient_{:04d}'.format(temp_inx)+'_slice_{:04d}'.format(current_slice)+'.jpg'))
                else:
                    aligned_image.save(os.path.join(target_save_path,'test_new', 'patient_{:04d}'.format(temp_inx)+'_slice_{:04d}'.format(current_slice)+'.jpg'))
                current_slice += 1               
            for current_no_mass_slice in range(len(selected_no_mass)):       
                                    
                slice_a = current_CT[:,:,selected_no_mass[current_no_mass_slice]] # slice images
                slice_b = current_prostate[:,:,selected_no_mass[current_no_mass_slice]]     
                assert(img_a.size == img_b.size)
                

                slice_a = slice_a/slice_a.max()*255


                #Cannot devide by b's max as it is 0, but it is not neccesary
                #slice_b = slice_b/slice_b.max()*255


                img_a = Image.fromarray(slice_a) # slice images
                img_b = Image.fromarray(slice_b)   
                                    
                aligned_image = Image.new("RGB", (img_a.size[0] * 2, img_a.size[1]))
                aligned_image.paste(img_a, (0, 0))
                aligned_image.paste(img_b, (img_a.size[0], 0))
                if (temp_inx)%10 in [0,1,2,3,4,5,6,7]:
                    aligned_image.save(os.path.join(target_save_path,'train_new', 'patient_{:04d}'.format(temp_inx)+'_slice_{:04d}'.format(current_slice)+'.jpg'))
             
                else:
                    aligned_image.save(os.path.join(target_save_path,'test_new', 'patient_{:04d}'.format(temp_inx)+'_slice_{:04d}'.format(current_slice)+'.jpg'))
                current_slice += 1




        print("patient "+str(temp_inx)+" completed")

/home/vasant/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:23: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.


patient 0 completed
patient 1 completed
patient 2 completed
patient 3 completed
patient 4 completed



patient 5 completed
patient 6 completed
patient 7 completed


FileNotFoundError: [Errno 2] No such file or directory: './datasets/prostate_segmentation/val_new/patient_0008_slice_0000.jpg'